<a href="https://colab.research.google.com/github/anuraagkansara/SecureNLP/blob/master/code/NER_bilstm_CRf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/anuraagkansara/SecureNLP.git
!pip install glove_python
!python -m spacy download en_core_web_md
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_vectors_web_lg


Cloning into 'SecureNLP'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 534 (delta 6), reused 0 (delta 0), pack-reused 512
Receiving objects: 100% (534/534), 5.51 MiB | 19.00 MiB/s, done.
Resolving deltas: 100% (43/43), done.
     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700322 sha256=3535c3c5650ee5b91f620345f01bc655f8b6d7c438dfb915fb689e0383659a9b
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python
     |████████████████████████████████| 95.4MB 84.6MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=fa4fae5e0a55350dbafa050cee76e7ea4db48c4a36069a423e27ebaa9e55ec97
  Stored in directory: /tmp/pip-ephem-wheel-cache-e7uqu_46/wheels/c1/2c/5f/

In [0]:
!pip install mosestokenizer
!pip install python-crfsuite

  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-cp36-none-any.whl size=49120 sha256=ec9f9ba879b99cd4c182da68a0684f7df62afbac37ef2a1908b305f7f4e3d8c0
  Stored in directory: /root/.cache/pip/wheels/a2/e7/48/48d5e0f9c0cd5def2dfd7cb8543945f906448ed1313de24a29
  Created wheel for uctools: filename=uctools-1.2.1-cp36-none-any.whl size=3992 sha256=6127e3c0af043c121e6cd5a872068c26839ef50aaff2eed1aa79fd0c743b3361
  Stored in directory: /root/.cache/pip/wheels/f3/97/7b/8e52dff3c5b4e8cd4553518850dc0f19f99a7d7ee99b541f60
  Created wheel for toolwrapper: filename=toolwrapper-1.0.0-cp36-none-any.whl size=3226 sha256=6c8051f6ead67670d773d1beea1a3fe8ba5b0a2747c6d4a277b9379de1aa5e25
  Stored in directory: /root/.cache/pip/wheels/5c/b0/2e/b8c414550c8372586ebaab634cf1f93733349cfe2b1d694fe8
Successfully built mosestokenizer uctools toolwrapper
     |████████████████████████████████| 757kB 4.9MB/s 


In [0]:
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from mosestokenizer import MosesDetokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import os
import nltk
import sklearn
import pycrfsuite
from sklearn.metrics import classification_report
import spacy
encore_model = spacy.load('en_core_web_md')

In [0]:
# Not including non relevant sentences in the dataset using this function (Undersampling for task 2)

def sentenceRead(path, sep):
    total_sentences = 0
    filenames = os.listdir(path)                    # gets all the files in the folder where data is present
    sentences = []                                  # initialize empty list named sentence
    for filename in filenames:
        with open(path + filename) as file:                               
            lines = file.read()                     # reads entire file from a folder
        lines = lines.split(sep)                    # split the sentence by newline
        for line in lines:              
            total_sentences += 1
            l = []                                  # inilialize empty list to hold the single lines
            for i in line.splitlines():             # for x in a single line of a sentence
                w = i.split(' ')                    # split x into two parts...... one having the word and another having the iob tag
                if w[0] == "''":                    # if we found no word on the line of a sentence, then make w[0] as "
                    w[0] = '"'                      # the data is such that after every sentence (full stop),
                if len(w) == 2:                     # we have only a newline character in a line, followed by a word in next line
                    l.append(np.array(w))           # only when we have a 'word' and a 'iob' tag in w (making len(w) == 2), we will append
            l = np.array(l)                         # converts entire list 'l' into numpy array
            if sum([x[1] == 'O' for x in l]) < len(l): # number of 'Os' found are less than length of the sentence, then add that sentence
              sentences.append(l)                         # to the list, since number of 'Os' are less means we have other iob tags too.
    return sentences,total_sentences                         # append  only those sentences which have tags other than 'O' as well


In [0]:
train,train_count = sentenceRead("SecureNLP/train/tokenized/", " \n")   #training data

In [0]:
train[0]

array([['The', 'O'],
       ['Equation', 'O'],
       ['group', 'O'],
       ...,
       ['797', 'O'],
       ['-', 'O'],
       ['8709', 'O']], dtype='<U75')

In [0]:
print(train_count)
print(len(train))
import plotly.graph_objects as go


# fig = go.Figure([go.Bar(x=sentences, y=[len(train),count - len(train)])])
# fig.show()


9505
2220


In [0]:
# test = []
# test_count = 0
# for i in [1, 2, 3]:
#     x, tc = sentenceRead(f"SecureNLP/test_{i}/tokenized/", "\n\n")
#     test_count += tc 
#     test.extend(x)  #adds another list to end of previous list

# There is no sentence in training and test data with all 'Os'. Removing the non relevant data from the dataset

# for i in test:
#   sum = 0
#   for x, y in i:
#     if y == 'O':
#       sum +=1
#   if sum == len(i):
#     print(x)
# test

In [0]:
ts = 0
sentencesout = []
tokenout = []
with open('SecureNLP/Task2.test') as fileout:                               
  linesout = fileout.read()                     # reads entire file from a folder              
linesout = linesout.split('\n\t\n')
# print(linesout[0])
for line in linesout:
  ts += 1
  l = []     
  t=[]                             # inilialize empty list to hold the single lines
  for i in line.splitlines():             # for x in a single line of a sentence
      w = i.split('\t')                    # split x into two parts...... one having the word and another having the iob tag
      if w[0] == "''":                    # if we found no word on the line of a sentence, then make w[0] as "
          w[0] = '"'                      # the data is such that after every sentence (full stop),
      if len(w) == 2:                     # we have only a newline character in a line, followed by a word in next line
          t.append(np.array(w[0]))
          l.append(np.array(w))           # only when we have a 'word' and a 'iob' tag in w (making len(w) == 2), we will append
  t = np.array(t)
  l = np.array(l)                         # converts entire list 'l' into numpy array
  if sum([x[1] == 'O' for x in l]) < len(l): # number of 'Os' found are less than length of the sentence, then add that sentence
    sentencesout.append(l) 
    tokenout.append(t)       

test = sentencesout
test_count = ts
# print(sentencesout)
print(len(test[0]))
tokenout[0]
sentencesout[0]

27


array([['Seaduke', 'O'],
       ['victims', 'O'],
       ['are', 'O'],
       ['generally', 'O'],
       ['first', 'O'],
       ['infected', 'O'],
       ['with', 'O'],
       ['Cozyduke', 'O'],
       ['and', 'O'],
       [',', 'O'],
       ['if', 'B-Modifier'],
       ['the', 'O'],
       ['computer', 'B-Entity'],
       ['appears', 'I-Entity'],
       ['to', 'I-Entity'],
       ['be', 'I-Entity'],
       ['a', 'I-Entity'],
       ['target', 'I-Entity'],
       ['of', 'I-Entity'],
       ['interest', 'I-Entity'],
       [',', 'O'],
       ['the', 'B-Entity'],
       ['operators', 'I-Entity'],
       ['will', 'O'],
       ['install', 'B-Action'],
       ['Seaduke', 'B-Entity'],
       ['.', 'O']], dtype='<U10')

In [0]:
# import codecs 
# testIn = codecs.open('Subtask1_2.in',encoding = 'utf-8')
# print(type(testIn))
# ts = 0
# with open('Subtask1_2.in') as file:                               
  # lines = file.read()                     # reads entire file from a folder
# with open('Task2.test') as fileout:                               
#   linesout = fileout.read()                     # reads entire file from a folder

# lines = lines.split('\n')
# lines = np.array(lines)
# linesout = linesout.split('\n')
# linesout = np.array(linesout)

# print(lines)
# print(linesout)

# for line in lines:              
#   ts += 1
#   l = []                                  # inilialize empty list to hold the single lines
#   for i in line.splitlines():


In [0]:
dev,dev_count = sentenceRead("SecureNLP/dev/tokenized/", "\n \n")         #reads the relevant sentences of the development set

In [0]:
from imblearn.over_sampling import SMOTE
# corpus = Corpus() 
# corpus.fit(embed, window=10)

# glove = Glove(no_components=100, learning_rate=0.05)
 
# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)
# glove.save('glove.model')



/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [0]:
sentences = ['Train', 'Development','Test']
# fig = go.Figure(go.Bar(x=sentences, y=[train_count,dev_count,test_count], name='Total'))
fig = go.Figure(go.Bar(x=sentences, y=[train_count - len(train),dev_count - len(dev),test_count-len(test)], name='Non-Relevant'))
fig.add_trace(go.Bar(x=sentences, y=[len(train),len(dev),len(test)], name='Relevant'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['Train','Development','Test']})
fig.show()

In [0]:
# Counting how many labels are there in total in train, dev and test set
count = {
    "train": 0,
    "dev":0,
    "test":0
    }
tag_labels = ["train", "dev", "test"]                       
for i, dataField in enumerate([train, dev, test]):      # takes the relevant sentences from all the three sets
    print(tag_labels[i])
    counts = {                                         # make a dictionary that could count number of IOB tags.
        "B-Action": 0, 
        "I-Action": 0, 
        "B-Entity": 0, 
        "I-Entity": 0, 
        "B-Modifier": 0, 
        "I-Modifier": 0,
        "O": 0,
        "total": 0
    }
    for singleSentence in dataField:
        for singleWord in singleSentence:
            counts[singleWord[1]] += 1                # doc is the sentence, word is a list in that sentence [word,iob]                                    
            counts["total"] += 1                # word[1] will be the iob tag and the dictionary will access the count of them 
    count[tag_labels[i]] = counts
    print(counts)


train
{'B-Action': 3535, 'I-Action': 796, 'B-Entity': 7562, 'I-Entity': 21354, 'B-Modifier': 2215, 'I-Modifier': 137, 'O': 79082, 'total': 114681}
dev
{'B-Action': 122, 'I-Action': 6, 'B-Entity': 254, 'I-Entity': 529, 'B-Modifier': 79, 'I-Modifier': 0, 'O': 1267, 'total': 2257}
test
{'B-Action': 125, 'I-Action': 9, 'B-Entity': 249, 'I-Entity': 499, 'B-Modifier': 79, 'I-Modifier': 0, 'O': 1320, 'total': 2281}


In [0]:
sentences = ['Train', 'Development','Test']
fig = go.Figure(go.Bar(x=sentences, y=[count['train']['I-Action'],count['dev']['I-Action'],count['test']['I-Action']], name='I-Action'))
fig.add_trace(go.Bar(x=sentences, y=[count['train']['B-Action'],count['dev']['B-Action'],count['test']['B-Action']], name='B-Action'))
fig.add_trace(go.Bar(x=sentences, y=[count['train']['B-Entity'],count['dev']['B-Entity'],count['test']['B-Entity']], name='B-Entity'))
fig.add_trace(go.Bar(x=sentences, y=[count['train']['I-Entity'],count['dev']['I-Entity'],count['test']['I-Entity']], name='I-Entity'))
fig.add_trace(go.Bar(x=sentences, y=[count['train']['B-Modifier'],count['dev']['B-Modifier'],count['test']['B-Modifier']], name='B-Modifier'))
fig.add_trace(go.Bar(x=sentences, y=[count['train']['I-Modifier'],count['dev']['I-Modifier'],count['test']['I-Modifier']], name='I-Modifier'))
fig.add_trace(go.Bar(x=sentences, y=[count['train']['O'],count['dev']['O'],count['test']['O']], name='O'))
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['Train','Development','Test']})
fig.show()

In [0]:
# TreebankWordDetokenizer converts tokens into sentences.

def detokenize(sent):
  sent = list(sent[:, 0])                   # Takes only the first column of and detokenizes it into a sentence
  d = TreebankWordDetokenizer()
  return d.detokenize(sent)

In [0]:
# MosesDetokenizer converts the tokens into a sentence, detokenize the sentence

# def detokenize(sent):                         
#     sent = list(sent[:, 0])                   # Takes only the first column of and detokenizes it into a sentence
#     with MosesDetokenizer('en') as detokenize:
#         sentence = detokenize(sent)
#     return sentence

In [0]:
# def word2features(sent, i):
#     word = sent[i][0]
#     postag = sent[i][1]
#     features = [
#         'bias',
#         'word.lower=' + word.lower(),
#         'word[-3:]=' + word[-3:],
#         'word[-2:]=' + word[-2:],
#         'word.isupper=%s' % word.isupper(),
#         'word.istitle=%s' % word.istitle(),
#         'word.isdigit=%s' % word.isdigit(),
#         'postag=' + postag,
#         'postag[:2]=' + postag[:2],
#         'len=' + str(len(word))
#     ]
#     if i > 0:
#         word1 = sent[i-1][0]
#         postag1 = sent[i-1][1]
#         features.extend([
#             '-1:word.lower=' + word1.lower(),
#             '-1:word.istitle=%s' % word1.istitle(),
#             '-1:word.isupper=%s' % word1.isupper(),
#             '-1:postag=' + postag1,
#             '-1:postag[:2]=' + postag1[:2],
#             'len=' + str(len(word1))
#         ])
#     else:
#         features.append('BOS')
        
#     if i < len(sent)-1:
#         word1 = sent[i+1][0]
#         postag1 = sent[i+1][1]
#         features.extend([
#             '+1:word.lower=' + word1.lower(),
#             '+1:word.istitle=%s' % word1.istitle(),
#             '+1:word.isupper=%s' % word1.isupper(),
#             '+1:postag=' + postag1,
#             '+1:postag[:2]=' + postag1[:2],
#             'len=' + str(len(word1))
#         ])
#     else:
#         features.append('EOS')
                
#     return features


# def sent2features(sent):
#     # doc = detokenize(sent)
#     return [word2features(sent, i) for i in range(len(sent))]

In [0]:
# We defined some features

def word2features(sent, i):                    
    w = sent[i]                                         #pick the first sentence
    features = {                                        #assign a some features to that sentence
        'bias': 1.0,
        'lowercase_word': w.lower_,                     # all these are defined token functions in spacy
        'word_last_3_chars': w.suffix_,
        'word_last_2_chars': w.text[-2:],
        'word_is_uppercase': w.is_upper,
        'word_is_digit': w.is_digit,
        'shape': w.shape_,
        'postag': w.pos_,
        'tag': w.tag_,
        'len': len(w)
    }
    if i > 0:                                            # all sentences except the first sentence
        w1 = sent[i-1]                                   # assign w1 to the previous sentence
        features.update({                                # assign features to the previous words as well  
            'previous_word_lower': w1.lower_,
            'previous_word_is_upppercase': w1.is_upper,
            'previous_word_postag': w1.pos_,
            'previous_word_tag': w1.tag_,
            'previous_word_shape': w1.shape_,
            'len_prev': len(w1)
        })
    else:                                                 # for the first sentence
        features['BOS'] = True                            # Beginning of sentence == true
    if i < len(sent)-1:                                   # all sentences except the last one
        w1 = sent[i+1]                                    # assign w1 to next sentence and assign features to next words as well
        features.update({
            'next_word_lower': w1.lower_,
            'next_word_is_upppercase': w1.is_upper,
            'next_word_postag': w1.pos_,
            'next_word_tag': w1.tag_,
            'next_word_shape': w1.shape_,
            'len_next': len(w1)
        })
    else:
        features['EOS'] = True                            # End of sentence == true
                
    return features                                       # features will contain a set of all the features which were defined above

def sent2features(sent):              # when I tested there is no affect of en_core_web_md on the data.
    doc = encore_model(detokenize(sent))       # Using moses detokenizer it detokenizes the sentence and then puts it in the spacy.load('en_core_web_md')
                                       # en_core_web_md adds the necessary features in the sentences which we will exploid in word2features
    return [word2features(doc, i) for i in range(len(doc))]       # 'en_core_web_md' is a pretrained model 
                                                                  # for every sentence it creates the feature vectors using the features above

In [0]:
def sent2labels(sent):                                    # if the iob tag is not 'O' then take only the entity,modifier and action, not
    return [y[2:] if y != 'O' else y for x, y in sent]    # the B,I part. If it is 'O' then take 'O' as such

def sent2tokens(sent):                                    # return only the word tokens of the sentence
    return [x for x, y in sent]

def get_all_labels(train_sents):                          # not called here in this notebook. Still lets look what is going on here
    result = []                                           # returns all the possible lables 
    for sent in train_sents:                              # After running this we got, {'Action', 'Entity', 'Modifier', 'O'}
        labels = sent2labels(sent)
        result.extend(labels)
        
    return set(result)

In [0]:
x = sent2features(train[0])
print(x)
# print(train[0])

# document = encore_model(detokenize(train[0]))
# docu = detokenize(train[0])
# print(docu)
# print(document)
# print(train[0])

[{'bias': 1.0, 'lowercase_word': 'the', 'word_last_3_chars': 'The', 'word_last_2_chars': 'he', 'word_is_uppercase': False, 'word_is_digit': False, 'shape': 'Xxx', 'postag': 'DET', 'tag': 'DT', 'len': 3, 'BOS': True, 'next_word_lower': 'equation', 'next_word_is_upppercase': False, 'next_word_postag': 'PROPN', 'next_word_tag': 'NNP', 'next_word_shape': 'Xxxxx', 'len_next': 8}, {'bias': 1.0, 'lowercase_word': 'equation', 'word_last_3_chars': 'ion', 'word_last_2_chars': 'on', 'word_is_uppercase': False, 'word_is_digit': False, 'shape': 'Xxxxx', 'postag': 'PROPN', 'tag': 'NNP', 'len': 8, 'previous_word_lower': 'the', 'previous_word_is_upppercase': False, 'previous_word_postag': 'DET', 'previous_word_tag': 'DT', 'previous_word_shape': 'Xxx', 'len_prev': 3, 'next_word_lower': 'group', 'next_word_is_upppercase': False, 'next_word_postag': 'NOUN', 'next_word_tag': 'NN', 'next_word_shape': 'xxxx', 'len_next': 5}, {'bias': 1.0, 'lowercase_word': 'group', 'word_last_3_chars': 'oup', 'word_last_2_c

In [0]:
# ************     FOR TRAINING DATA       ****************
X_train = []                                
y_train = []
for i, sent in enumerate(train):
    if len(sent) == 0:
      continue
    x = sent2features(sent)                # each sent is a numpy array of two cols --> {word,iob tag}
    y = sent2labels(sent)                  # get the labels without the prefix IOB tags.
    if len(x) == len(y):                   # for every sentence the number of words should be same as number of tags 
        X_train.append(x)
        y_train.append(y)

# ***************       FOR TEST DATA    ****************
rejected_test = []                         
X_test = []
y_test = []
y_test_original = []
print(len(test))
for i, sent in enumerate(test[:-1]):
    # sent = list(sent[:, 0]) 
    # print(i) 
    if len(sent) == 0:
      continue
    x = sent2features(sent)                # same thing that is done for test data
    y = sent2labels(sent)
    y_test_original.append([y for x, y in sent])    # y_test_original will contain list of IOB tags for all test sentences
    if len(x) == len(y):                   # for every sentence the number of words should be same as number of tags                         
        X_test.append(x)
        y_test.append(y)                   # Append only the labels without the prefix I ,B   
    else:
        rejected_test.append((i, sent))    # if (# words != # tags) in one sentence 
        
# *************   DEVELOPMENT SET     ******************
rejected_dev = []                          
X_dev = []                                 # Do similar thing with develpment set as you did with test set
y_dev = []
y_dev_original = []
for i, sent in enumerate(dev):
    if len(sent) == 0:
      continue
    x = sent2features(sent)
    y = sent2labels(sent)
    y_dev_original.append([y for x, y in sent])
    if len(x) == len(y):
        X_dev.append(x)
        y_dev.append(y)
    else:
        rejected_dev.append((i, sent))

90


In [0]:
# corpus = Corpus() 
# corpus.fit(embed, window=10)

# glove = Glove(no_components=100, learning_rate=0.05)
 
# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)
# glove.save('glove.model')


# sm = SMOTE(random_state = 2) 
# X_train_res, y_train_res = sm.fit_sample(X_train, y_train) 

In [0]:
trainer = pycrfsuite.Trainer(verbose=False)     # To train the model, we create pycrfsuite.Trainer, load the training data
                                                # and call 'train' method. Create pycrfsuite.Trainer and load the training data to CRFsuite
for xseq, yseq in zip(X_train, y_train):        # zip makes the correspondance between say 1st training example's X with 1st training examples y
    trainer.append(xseq, yseq)                  # appends the x and y pairs in the trainer

# Set the training parameters   
trainer.set_params({                            # We will use L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.
    'c1': 1.0,                                  # coefficient for L1 penalty 
    'c2': 1e-2,                                 # coefficient for L2 penalty
    'max_iterations': 400,                      # stop earlier
    'feature.possible_transitions': True,        # include transitions that are possible, but not observed
    'epsilon':1e-5,
    
})                                              # Taken it directly from the example on CoNLL dataset for NER
trainer.train('conll2002-esp.crfsuite')         # train the model on this crfsuite
example_sent = test[0]                          # Take the example sentence
tagger = pycrfsuite.Tagger()                    # To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method
tagger.open('conll2002-esp.crfsuite')
print('\nExample Sentence\n')
print(' '.join(sent2tokens(example_sent)), end='\n\n') # Just to print the example Sentence

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))



Example Sentence

Seaduke victims are generally first infected with Cozyduke and , if the computer appears to be a target of interest , the operators will install Seaduke .

Predicted: Entity Entity O O O O O O O O O O O O O O O O O O O Entity Entity O Action Entity O
Correct:   O O O O O O O O O O Modifier O Entity Entity Entity Entity Entity Entity Entity Entity O Entity Entity O Action Entity O


In [0]:
trainer.params()              # set of possible trainer parameters, these are default

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [0]:
def addingIB(pred):                         # Now since we have the resulting labels without the I,B 
    result = []                                       # We will add I,B before the predicted tags      
    for prediction in pred:                     
        b = True
        current_token = ''
        temp = []
        for token in prediction:                      # for a single prediction of a sentence (this will be a list),
            if token == 'O':                          # take each token and see if its 'O' or not 'O'
                temp.append(token)                # for 'O', do nothing
                if not b:                     
                    b = True
            elif token != 'O':                        # if its not 'O', then if its in the beginning, 
                if b:                         # give the beginning tag to already existing label (Entity, Modifier or Attribute) 
                    b = False
                    current_token = token
                    temp.append('B-' + token)
                elif token == current_token:          # if previous token and current token is same then do 'I-' else do 'B-'
                    temp.append('I-' + token)
                else:
                    current_token = token
                    temp.append('B-' + token)
        result.append(temp)                       # Append all these labeled IOB sentences to result and return the result
    return result 

In [0]:
test[0]

array([['Seaduke', 'O'],
       ['victims', 'O'],
       ['are', 'O'],
       ['generally', 'O'],
       ['first', 'O'],
       ['infected', 'O'],
       ['with', 'O'],
       ['Cozyduke', 'O'],
       ['and', 'O'],
       [',', 'O'],
       ['if', 'B-Modifier'],
       ['the', 'O'],
       ['computer', 'B-Entity'],
       ['appears', 'I-Entity'],
       ['to', 'I-Entity'],
       ['be', 'I-Entity'],
       ['a', 'I-Entity'],
       ['target', 'I-Entity'],
       ['of', 'I-Entity'],
       ['interest', 'I-Entity'],
       [',', 'O'],
       ['the', 'B-Entity'],
       ['operators', 'I-Entity'],
       ['will', 'O'],
       ['install', 'B-Action'],
       ['Seaduke', 'B-Entity'],
       ['.', 'O']], dtype='<U10')

In [0]:
 # *************    Evaluate the Model      *********

y_test_pred = [tagger.tag(test) for test in X_test]    
labels = {'Modifier': 0, 'Entity': 1, 'Action': 2,'O': 3}     # create a dictionary mapping labels to numbers

predictions = np.array([labels[tag] for row in y_test_pred for tag in row]) # whatever predictions we got, take the predictions of each
                                                                  # sentence and check for each tag in a sentence and map that tag to a number
                                                                  # Just doing the mapping from tag to number of predictions
truths = np.array([labels[tag] for row in y_test for tag in row]) # Just doing the mapping from tag to number of actual true values

print(classification_report(                                      # Classification report without I,B tags
    truths, predictions,
    target_names=['Modifier', 'Entity', 'Action', 'O']))

y_test_pred = addingIB(y_test_pred)                            # adds the suffix of B and I to tags
for rejected in rejected_test:                                    # Push all the rejected sentences with all 'O' 
    y_test_pred.insert(rejected[0], ['O']*len(rejected[1]))
                                                                  # Map all the possible tags to numbers
labels_full = {'I-Modifier': 0, 'I-Entity': 1, 'B-Modifier': 2, 'B-Action': 3, 'B-Entity': 4, 'I-Action': 5, 'O': 6} 

predictions_full = np.array([labels_full[tag] for row in y_test_pred for tag in row])
truths_full = np.array([labels_full[tag] for row in y_test_original for tag in row])



print(classification_report(                                       # Classification report with I,B tags
    truths_full, predictions_full,
    target_names=['I-Modifier', 'I-Entity', 'B-Modifier', 'B-Action', 'B-Entity', 'I-Action', 'O']))

              precision    recall  f1-score   support

    Modifier       0.59      0.70      0.64        67
      Entity       0.61      0.69      0.65       650
      Action       0.57      0.65      0.61       118
           O       0.78      0.70      0.74      1190

    accuracy                           0.69      2025
   macro avg       0.64      0.69      0.66      2025
weighted avg       0.70      0.69      0.70      2025

              precision    recall  f1-score   support

  I-Modifier       0.00      0.00      0.00         0
    I-Entity       0.51      0.56      0.53       497
  B-Modifier       0.60      0.58      0.59        79
    B-Action       0.53      0.46      0.49       124
    B-Entity       0.27      0.21      0.23       247
    I-Action       0.04      0.11      0.05         9
           O       0.73      0.73      0.73      1316

    accuracy                           0.61      2272
   macro avg       0.38      0.38      0.38      2272
weighted avg       0.61

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [0]:
print(y_test_pred[0])
tokenout[0]

['B-Entity', 'I-Entity', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Entity', 'I-Entity', 'O', 'B-Action', 'B-Entity', 'O']


array(['Seaduke', 'victims', 'are', 'generally', 'first', 'infected',
       'with', 'Cozyduke', 'and', ',', 'if', 'the', 'computer', 'appears',
       'to', 'be', 'a', 'target', 'of', 'interest', ',', 'the',
       'operators', 'will', 'install', 'Seaduke', '.'], dtype='<U9')

In [0]:
outputPred = open('Task2.out','w+')
ols = []

for word,label in zip(tokenout,y_test_pred):
  # if len(word)!=len(label):
  #     # print(word)
  #     # print("Word Length: ",len(word))
  #     # print("label Length: ",len(label))
  #   for i in range(len(word)):
  #     ols.append(word[i]+'\t'+'O'+'\n')
  # else:
  for i in range(len(word)):
    ols.append(word[i]+'\t'+label[i]+'\n')
    # print(word[i]+'\t'+label[i]+'\n')
  ols.append('\n')
print(ols)
# ols = ols[:-2]
print(len(ols))
ols = ''.join(ols)
outputPred.write(ols)


['Seaduke\tB-Entity\n', 'victims\tI-Entity\n', 'are\tO\n', 'generally\tO\n', 'first\tO\n', 'infected\tO\n', 'with\tO\n', 'Cozyduke\tO\n', 'and\tO\n', ',\tO\n', 'if\tO\n', 'the\tO\n', 'computer\tO\n', 'appears\tO\n', 'to\tO\n', 'be\tO\n', 'a\tO\n', 'target\tO\n', 'of\tO\n', 'interest\tO\n', ',\tO\n', 'the\tB-Entity\n', 'operators\tI-Entity\n', 'will\tO\n', 'install\tB-Action\n', 'Seaduke\tB-Entity\n', '.\tO\n', '\n', 'In\tO\n', 'the\tO\n', 'months\tO\n', 'that\tO\n', 'followed\tO\n', ',\tO\n', 'the\tB-Entity\n', 'Duke\tI-Entity\n', 'group\tI-Entity\n', 'began\tO\n', 'to\tO\n', 'target\tB-Action\n', 'victims\tB-Entity\n', 'with\tB-Modifier\n', '€\tB-Entity\n', 'œOffice\tI-Entity\n', 'Monkeys\tI-Entity\n', '€\tI-Entity\n', '\u200b-\tI-Entity\n', 'and\tI-Entity\n', '€\tI-Entity\n', 'œeFax\tI-Entity\n', '€\tI-Entity\n', '\u200b-themed\tI-Entity\n', 'emails\tI-Entity\n', ',\tO\n', 'booby\tO\n', '-\tO\n', 'trapped\tO\n', 'with\tO\n', 'a\tO\n', 'Cozyduke\tB-Entity\n', 'payload\tI-Entity\n', '.

24136

In [0]:
# Do the same as above for development set as well
y_dev_pred = [tagger.tag(dev) for dev in X_dev]
labels = {'Modifier': 0, 'Entity': 1, 'Action': 2,'O': 3}

predictions = np.array([labels[tag] for row in y_dev_pred for tag in row])
truths = np.array([labels[tag] for row in y_dev for tag in row])

print(classification_report(                              # Without I,B
    truths, predictions,
    target_names=['Modifier', 'Entity', 'Action', 'O']))

y_dev_pred = addingIB(y_dev_pred)
for rejected in rejected_dev:
    y_dev_pred.insert(rejected[0], ['O']*len(rejected[1]))

labels_full = {'I-Modifier': 0, 'I-Entity': 1, 'B-Modifier': 2, 'B-Action': 3, 'B-Entity': 4, 'I-Action': 5, 'O': 6}

predictions_full = np.array([labels_full[tag] for row in y_dev_pred for tag in row])
truths_full = np.array([labels_full[tag] for row in y_dev_original for tag in row])

# print('predictions_full\t',predictions_full.shape)
# print('truths_full\t',truths_full.shape)

print(classification_report(                               # With I,B   # Dev set has no I-modifier so try to skip checking that
    truths_full, predictions_full,
    target_names=['I-Modifier','I-Entity', 'B-Modifier', 'B-Action', 'B-Entity', 'I-Action', 'O']))    # I-modifier f1 score is 0 since there is no I-modifier in dev set

              precision    recall  f1-score   support

    Modifier       0.50      0.56      0.53        77
      Entity       0.53      0.68      0.60       761
      Action       0.62      0.68      0.65       126
           O       0.73      0.59      0.65      1251

    accuracy                           0.62      2215
   macro avg       0.59      0.63      0.61      2215
weighted avg       0.65      0.62      0.63      2215

              precision    recall  f1-score   support

  I-Modifier       0.00      0.00      0.00         0
    I-Entity       0.42      0.60      0.49       529
  B-Modifier       0.54      0.53      0.54        79
    B-Action       0.60      0.57      0.59       122
    B-Entity       0.29      0.26      0.27       254
    I-Action       0.00      0.00      0.00         6
           O       0.71      0.59      0.65      1267

    accuracy                           0.55      2257
   macro avg       0.37      0.36      0.36      2257
weighted avg       0.58

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [0]:
# Let's check what classifier learned

from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(10))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-10:])

Top likely transitions:
O      -> O       3.038400
Entity -> Entity  2.404593
Modifier -> Modifier 1.667121
Action -> Action  0.632785
Action -> Modifier 0.198362
O      -> Action  0.062804
Modifier -> Entity  -0.134302
Modifier -> O       -0.268724
Entity -> Action  -0.707597
Entity -> Modifier -0.770455

Top unlikely transitions:
Modifier -> Entity  -0.134302
Modifier -> O       -0.268724
Entity -> Action  -0.707597
Entity -> Modifier -0.770455
Entity -> O       -0.999484
Action -> O       -1.450335
O      -> Entity  -1.510409
Action -> Entity  -1.812161
O      -> Modifier -2.330107
Modifier -> Action  -6.952589


In [0]:
# Checking the state features

def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(10))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-10:])

Top positive:
5.938542 Modifier lowercase_word:using
4.623100 Action lowercase_word:encrypts
4.413275 Modifier lowercase_word:based
4.306546 Modifier lowercase_word:according
4.257330 O      EOS
4.226019 Entity previous_word_lower:beacons
4.097488 O      tag:MD
4.084283 Modifier lowercase_word:every
3.950359 Action lowercase_word:stores
3.581964 Action next_word_lower:engineer

Top negative:
-2.347260 Entity lowercase_word:case
-2.419966 Entity previous_word_lower:disk
-2.520221 O      previous_word_lower:observed
-2.603409 Action lowercase_word:have
-2.621822 Modifier postag:VERB
-2.647514 Entity tag:VBD
-2.856954 Entity tag:VBZ
-3.090989 Entity tag:WDT
-3.121717 Entity tag:TO
-3.633842 Entity lowercase_word:beacons


# **The below is the example of NER using PYTORCH. This is no way related to the above code of Task II.**

In [0]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [0]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [0]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [0]:
training_data,train_count = sentenceRead("SecureNLP/train/tokenized/", " \n")
total = []
for sent in training_data:
  p = [x for x, y in sent]  
  q = [y for x, y in sent]  
  tup = (p,q)
  total.append(tup)


In [0]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
# Make up some training data
training_data = total
print((training_data[0]))
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]])
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))

(['The', 'Equation', 'group', 'is', 'a', 'highly', 'sophisticated', 'threat', 'actor', 'that', 'has', 'been', 'engaged', 'in', 'multiple', 'CNE', '(', 'computer', 'network', 'exploitation', ')', 'operations', 'dating', 'back', 'to', '2001', ',', 'and', 'perhaps', 'as', 'early', 'as', '1996', '.', 'The', 'Equation', 'group', 'uses', 'multiple', 'malware', 'platforms', ',', 'some', 'of', 'which', 'surpass', 'the', 'well', '-', 'known', '"', 'Regin', '"', 'threat', 'in', 'complexity', 'and', 'sophistication', '.', 'The', 'Equation', 'group', 'is', 'probably', 'one', 'of', 'the', 'most', 'sophisticated', 'cyber', 'attack', 'groups', 'in', 'the', 'world', ';', 'and', 'they', 'are', 'the', 'most', 'advanced', 'threat', 'actor', 'we', 'have', 'seen', '.', 'We', 'call', 'this', 'threat', 'actor', 'the', 'Equation', 'group', 'because', 'of', 'their', 'love', 'for', 'encryption', 'algorithms', 'and', 'obfuscation', 'strategies', 'and', 'the', 'sophisticated', 'methods', 'used', 'throughout', 'th

KeyError: ignored